# Road Safety data
Data source: https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data

## Data Preparation

In [1]:
import os
import pandas as pd
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
import plotly
import plotly.express as px
import datetime

# Change the current working directory, comment if data is in the same directory
#os.chdir('data')
# Print the current working directory
#print("Current working directory: {0}".format(os.getcwd()))

accident = pd.read_csv('dft-road-casualty-statistics-accident-last-5-years.csv', sep=",", low_memory=False)
casualty = pd.read_csv('dft-road-casualty-statistics-casualty-last-5-years.csv', sep=",", low_memory=False)
vehicle = pd.read_csv('dft-road-casualty-statistics-vehicle-last-5-years.csv', sep=",", low_memory=False)
xlsx = pd.read_excel('Road-Safety-Open-Dataset-Data-Guide.xlsx')

#accident.info(verbose = True, show_counts = True)
#casualty.info(verbose = True, show_counts = True)
#vehicle.info(verbose = True, show_counts = True)

# Joining datasets
df_merged = vehicle.merge(casualty, how='outer', left_on=['accident_index','accident_year','accident_reference','vehicle_reference'], right_on=['accident_index','accident_year','accident_reference','vehicle_reference'])
df_merged = df_merged.merge(accident, how='outer', left_on=['accident_index','accident_year','accident_reference'], right_on=['accident_index','accident_year','accident_reference'])
#df.info(verbose = True, show_counts = True)
# Which fields are null now?
#print(df_merged.isnull().sum().loc[lambda s: s > 0])
df_merged = df_merged.fillna(-1)
# The only table that contains null values is accident
#accident.isnull().sum().loc[lambda s: s > 0]
print("Data from " + str(accident['date'].min()) + " to " + str(accident['date'].max()))       

Data from 01/01/2016 to 31/12/2020


In [2]:
# Filtering relevant fields
df_filtered = df_merged.loc[:,['accident_year', 'day_of_week', 'date', 'time',
    'longitude', 'latitude', 'local_authority_ons_district', 
    
    # Driver
    'vehicle_type', 'propulsion_code',
    'vehicle_manoeuvre', 'junction_location',
    'sex_of_driver', 'age_band_of_driver', 
    
    # Casualty
    'sex_of_casualty', 'age_band_of_casualty',
    'accident_severity', 'casualty_severity', 
    'casualty_class', 'casualty_type', 'car_passenger',     
    'casualty_home_area_type', 
    
    # Road
    'road_type', 'first_road_class', 'second_road_class',
    'junction_detail', 'junction_control',
    'urban_or_rural_area', 
    'did_police_officer_attend_scene_of_accident',
    'number_of_casualties', 'age_of_driver', 'age_of_vehicle', 'age_of_casualty', 'number_of_vehicles', 'speed_limit' #needs data prep
    # 'accident_index', 'accident_reference','vehicle_reference', 'casualty_reference', # indexes
    #'vehicle_direction_from', 'vehicle_direction_to', 'driver_imd_decile', 'casualty_imd_decile', # limited applicability
    # 'driver_home_area_type', , 'generic_make_model' # limited applicability
    # 'local_authority_district', 'local_authority_highway', 'police_force', 'lsoa_of_accident_location' # Repeated
]]

# adding EV attribute
#['Electric', 'Hybrid electric', 'Electric diesel', 'New fuel technology']
df_filtered['ev'] = df_filtered['propulsion_code'].apply(lambda x: 'EV' if (x in [3,8,12,10]) else 'non-EV')

# Time conversion
df_filtered['time'] = pd.to_datetime(df_filtered['time'], errors='coerce').dt.round(freq='H').astype(str).str[-8:]

# Month
df_filtered['date'] = df_filtered['date'].apply(lambda x: datetime.datetime.strptime(x, "%d/%m/%Y"))
df_filtered['month'] = df_filtered['date'].apply(lambda x: str(x.year)+'-'+str(x.month).zfill(2))

In [3]:
# Importing geographical data
# Data source of geopgraphical data:
#https://mangomap.com/pgager2/maps/36876/United-Kingdom-Administrative-Boundaries-for-download-#
#https://ukdataservice.ac.uk/learning-hub/census/
#https://borders.ukdataservice.ac.uk/easy_download.html
#gdf = gpd.read_file("infuse_dist_lyr_2011.shp")
#gdf = gdf[['geo_code', 'name', 'geometry']]
gdf = gpd.read_file("UK_DISTRICTS_COUNTIES_CENSUS2011.shp")
gdf = gdf[['LAD11CD','LAD11NM','geometry']]
gdf.columns = ['geo_code', 'name', 'geometry']
#gdf.explore(legend=False)

# Finding Centroid
crs = 'GCS_WGS_1984'
gdf.to_crs(crs, inplace = True)
gdf.set_crs(crs, inplace = True)
gdf['centroid'] = gdf.centroid
gdf['centroid'] = gdf['centroid'].to_crs(crs)
gdf['centroid'] = gdf['centroid'].set_crs(crs)
gdf['longitude'] = gdf['centroid'].x
gdf['latitude'] = gdf['centroid'].y

C:\Users\camil\AppData\Local\Temp\ipykernel_12952\4034097126.py:17: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.centroid


In [4]:
attributes_dict = {
    'accident_year': 'quantitative',
    'age_of_driver': 'quantitative',
    'age_of_vehicle': 'quantitative',
    'age_of_casualty': 'quantitative',
    'number_of_vehicles': 'quantitative',
    'number_of_casualties': 'quantitative',
    'speed_limit': 'quantitative',
    'time': 'quantitative',
    'month':'quantitative',
    'ev': 'quantitative',
    
    'day_of_week': 'ordinal',    
    'age_band_of_driver': 'ordinal',    
    'age_band_of_casualty': 'ordinal',
    
    'vehicle_type': 'categorical',  
    'propulsion_code': 'categorical',  
    'vehicle_manoeuvre': 'categorical',  
    'junction_location': 'categorical', 
    'car_passenger': 'categorical', 
    'road_type': 'categorical',
    'junction_detail': 'categorical', 
    'junction_control': 'categorical',
    'second_road_class': 'categorical', 
    'first_road_class': 'categorical',    
    'casualty_type': 'categorical',
    
    'casualty_class': 'categorical',  
    'casualty_home_area_type': 'categorical', 
    'accident_severity': 'categorical', 
    'casualty_severity': 'categorical', 
    'sex_of_driver': 'categorical', 
    'sex_of_casualty': 'categorical', 
    'urban_or_rural_area': 'categorical', 
    'did_police_officer_attend_scene_of_accident': 'categorical',
    'local_authority_ons_district': 'categorical'
}

In [5]:
def substitute_by_label(field_name = 'local_authority_ons_district', df = df_filtered, xlsx = xlsx):
    ''' substitutes the field_name column to its correspondence accorsing to the xlsx
    '''
    columns = ['code/format','label']
    # Spreadhseet with the code to label correspondence
    xlsx_df = xlsx[xlsx['field name'] == field_name][columns].drop_duplicates(subset='label',keep='first')
    df_labeled = df.merge(xlsx_df, how='left', left_on=[field_name], right_on=['code/format'])
    df_labeled[field_name] = df_labeled['label']
    df_labeled.drop(columns=columns, inplace= True)
    return df_labeled

#df_labeled = substitute_by_label('local_authority_ons_district', df_filtered, xlsx)

# For all categorical and ordinal attributes
df_labeled = df_filtered
df_labeled.loc[:,'geo_code'] = df_filtered.loc[:,'local_authority_ons_district']
for key, value in attributes_dict.items():
    if value != 'quantitative':
        df_labeled = substitute_by_label(key, df_labeled, xlsx)
        
df_labeled.fillna('Data missing', inplace=True)        
districts = df_labeled['local_authority_ons_district'].unique()

In [6]:
def address_from_coordinates(top = 10, df_labeled = df_labeled):
    ''' Gets the 'top' amount of addresses for the coordinates in df_labeled
    Warning: this function can take quite a log time to process.
    '''
    # Remove other columns
    df_address = df_labeled.groupby(['local_authority_ons_district', 'latitude', 'longitude'], as_index = False)['number_of_casualties'].sum()
    df_address = df_address.sort_values(by=['local_authority_ons_district','number_of_casualties'], ascending = False)
    df_address = df_address.groupby('local_authority_ons_district').head(top).reset_index(drop=True)
    df_address['Address'] = df_address['latitude'].apply(lambda x: str(x))+', '+ df_address['longitude'].apply(lambda x: str(x))
    geolocator = Nominatim(user_agent="http")
    df_address['Address'] = df_address['Address'].apply(lambda x: geolocator.reverse(x))
    return df_address

# This takes a very long time for the full dataset (apx. 40min) so the file is pre-saved.
#df_address = address_from_coordinates(10, df_labeled = df_labeled)
#df_address.to_csv('df_address.csv')

# Importing addresses
df_address = pd.read_csv('df_address.csv')
df_address = df_address[['local_authority_ons_district', 'Address', 'number_of_casualties']]
#df_address

In [7]:
missing = ['Unknown', 'Not known', 'Other/Not known', 'Other','Undefined', 'Data missing or out of range', 'Data missing', 'Unclassified', 'Unallocated', 'unknown (self reported)', 'Unknown vehicle type (self rep only)', 'Other vehicle', -1]
    
def make_frequency_chart(field_name = 'junction_control', radio_totals = 'Exclude', df_chart = df_labeled, xlsx=xlsx, attributes_dict = attributes_dict, missing = missing):
    ''' For the df_chart, according to the data type in attributes_dict makes a frequency chart of the field_name
    Categories <=3: Pie chart
    Ordered: Line
    categorical: horizonal bar chart
    
    xlsx is the file containing the corespondence of names
    missing is the list of data missing names to be removed if radio_totals = 'Exclude'

    '''
    field_class = attributes_dict[field_name]
    title='Casualties: '+field_name.replace('_',' ').capitalize()
    hover_data = ['Percentage']
        
    # Frequency
    frequencies = pd.DataFrame(df_chart[field_name].unique(),columns=[field_name]).sort_values(by=field_name, ascending=True, ignore_index = True)
    for index, row in frequencies.iterrows():
        frequencies.loc[index,'Casualties'] = df_chart[df_chart[field_name]==row[0]]['number_of_casualties'].sum()
    if radio_totals == 'Exclude':
        frequencies = frequencies[~frequencies[field_name].isin(missing)]
    
    # Percentage    
    frequencies['Percentage'] = round(frequencies['Casualties']/frequencies['Casualties'].sum()*100,1)
    frequencies['Percentage'] = frequencies['Percentage'].apply(lambda x: str(x)+'%')  
    
    # Ordered (Ordinal) preparation
    if field_class == 'ordinal':
        # to read the xls with the correspondencies
        columns = ['code/format','label']
        # Spreadhseet with the code to label correspondence
        xlsx_df = xlsx[xlsx['field name'] == field_name][columns].drop_duplicates(subset='label',keep='first')
        frequencies = frequencies.merge(xlsx_df, how='left', left_on=[field_name], right_on=['label'])
        frequencies.sort_values(by=['code/format'], inplace=True)
    
    # Pie chart
    if frequencies.shape[0] <= 3:
        fig = px.pie(frequencies, values='Casualties', names=field_name, title=title)
        return fig #, map_full
        
    # Ordinal and quantitative
    if field_class != 'categorical':
        fig = px.line(frequencies, x=field_name, y='Casualties', hover_name = field_name, hover_data = hover_data, title = title)
        return fig #, map_full
    
    # Categorical
    if field_class == 'categorical':
        frequencies.sort_values(by='Casualties', ascending=True, inplace = True)
        fig = px.bar(frequencies, x='Casualties', y=field_name, hover_name = field_name, hover_data = hover_data, title = title)
        return fig #, map_full
    
#field_name = 'junction_control' #'time' #'day_of_week' #'propulsion_code' #'ev' 
#fig = make_frequency_chart(field_name)
#fig

In [8]:
# Removing from the options
del attributes_dict['local_authority_ons_district']

In [40]:
# For the Map hover data
#'junction_location': lambda x: pd.Series.mode(x)[0],
#'junction_control': lambda x: pd.Series.mode(x)[0],
#'first_road_class': lambda x: pd.Series.mode(x)[0],
#'second_road_class': lambda x: pd.Series.mode(x)[0],
#'did_police_officer_attend_scene_of_accident': lambda x: pd.Series.mode(x)[0],
    
aggregations = {
    'number_of_casualties':'sum',
    'speed_limit': 'mean',
    'road_type': lambda x: pd.Series.mode(x)[0],
    'junction_detail': lambda x: pd.Series.mode(x)[0],
    'urban_or_rural_area': lambda x: pd.Series.mode(x)[0],
    'vehicle_manoeuvre': lambda x: pd.Series.mode(x)[0]
}

hover_data = list(aggregations.keys())

## Dash app
Learned a lot from the [Dash tutorial](https://dash.plotly.com/layout)

Check the app at http://127.0.0.1:2077/

In [41]:
from dash import Dash, dcc, html, Input, Output, callback, dash_table
from jupyter_dash import JupyterDash
#from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc

app = JupyterDash(__name__, external_stylesheets = [dbc.themes.BOOTSTRAP], suppress_callback_exceptions=True)

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='homepage')
])

### Index page ###

index_page = html.Div([
    html.H1('UK Road Safety data'),
    dcc.Link('Go to the Road Casualties Map', href='/map'),
    html.Br(),
    dcc.Link('Go to the Attributes correlation analysis', href='/attributes'),
    dcc.Markdown('''This app allows an interactive analysis on the [UK Road Safety Data](https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data) from 01/01/2016 to 31/12/2020.
    
On the [Road Casualties Map](/map), first there is an overview of the full UK following the rule of thumb "Overview First, Zoom and Filter, Detail on Demand". 
Another reason for this is because plotting all the precise locations of casualties made the map very cluttered, losing a clear view of the bigger picture. 
This also greatly impacted the responsiveness of the visualisation, since less dots needed to be plotted. As an intteractive tool, it is possible to:    
* **Choose Map aggregation:**
    * General: Overview of in which districts the most road casualties happen in the UK. By clicking on a district you can see a table underneath with the top 10 addresses with the most casualties.
        *  Hovering over a point allows more information about it such as 'number_of_casualties', 'speed_limit', 'road_type', 'junction_location', 'junction_detail', 'junction_control', 'first_road_class', 'second_road_class', 'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident', 'vehicle_manoeuvre'
    * Precise: Pinpoint the exact roads where most accidents happen in the chosen district.
 
* **Filter:**
    * Accident severity level between Slight, Serious and Fatal according to the [Severity adjustment figure guidance 2020](https://data.dft.gov.uk/road-accidents-safety-data/severity-adjustment-figure-guidance-2020.docx)
    * Filter between EV and non-EV vehicles
        * EV vehicles are the ones with propulsion code in 'Electric', 'Hybrid electric', 'Electric diesel', 'New fuel technology'
    * If no option is selected, the filter is the same as selecting all categories.

On the right side, there is an accompanying attribute chart of the total number of casualties that is synched with the above mentioned filters. 
It is heavily based on the concepts by [Munzner, 2014](https://www.cs.ubc.ca/~tmm/vadbook/) for what idiom should be used according to the attribute type:
* Quantitative attributes ('accident_year', 'age_of_driver', 'age_of_vehicle', 'age_of_casualty', 'number_of_vehicles', 'number_of_casualties', 'speed_limit')
    * Idiom chosen: *line charts*
    * length is one of the most effective magnitude channels for quantitative values. The marks used for this are lines, because they make it easier to find trends, for example.
    * The x-axis is the value of the attribute
* Ordinal attributes ('day_of_week',  'year-month',  'time' (rounded to the hour), 'age_band_of_driver', 'age_band_of_casualty')
    * In a similar manner as the quantitative attributes, the x-axis presents the attribute in the correct order, with it's respective name.
* Categorical attributes
    * Most attributes on the dataset are categorical. These can be subdivided for easier understading as:
        * The driver and their vehicle: 'sex_of_driver', 'vehicle_type', 'propulsion_code', 'ev', 'vehicle_manoeuvre', 'car_passenger', 'accident_severity'
        * The casualty: 'casualty_severity', 'casualty_class', 'casualty_type', 'sex_of_casualty', 'casualty_home_area_type'
        * The road: 'road_type', 'first_road_class', 'second_road_class', 'junction_location', 'junction_control', 'junction_detail', 'urban_or_rural_area', 'did_police_officer_attend_scene_of_accident'
    * About the idiom
        * Idiom chosen: *Horizontal bar charts*
        * Bars are used instead of lines because of the expressiveness principle.
        * Horizontal  bars for legibility of the categories
    * Ordered by quantity for easier comparison between categories as well as easier look up at the highest category
    * Since all attributes sum up to the total amount of casualties, in case there are less than five categories, the idiom chosen is a *pie chart* for faster Part-to-whole judgement. In this case the marks used are Separate colored areas with the channels: Color for categorical attribute; Angle for quantitative attribute
* Some attributes have an extensive amount of unknown or missing information. 
For full data transparency, it is possible to choose between 'All data' or 'Exclude' the data that is declared as 'Unknown', 'Undefined', 'Data missing or out of range'

More information about these attributes can be found at [Road Safety Open Dataset Data Guide](https://data.dft.gov.uk/road-accidents-safety-data/Road-Safety-Open-Dataset-Data-Guide.xlsx)
'''),

    dcc.Markdown('''Authors: Camila Matoba and Sietske Wijffels
    
License of the data: [Open Government Licence v3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/)'''),
])



### Page 1 ###
# Data
df = df_labeled

# Parameters
# EV Options
ev_options = df['ev'].unique()
# accident_severity Options
accident_severity_options = df['accident_severity'].unique()
# Map aggregation options
all_options = {
    'General': ['UK'],
    'Precise': districts
}

# Map
map_full = px.scatter_mapbox(df, lat="latitude", lon="longitude", color='number_of_casualties', size='number_of_casualties', mapbox_style = "open-street-map",
                          height = 700, zoom = 5, opacity = 0.9, hover_data=['number_of_casualties'], color_continuous_scale=plotly.colors.sequential.Purp)    
map_full.update_layout(clickmode='event+select')

# App layout
page_1_layout = html.Div(id='full', children=[
    html.H1('Road Casualties Map'),
    
    html.Div([
        # Left
        html.Div([
            html.Label('Map aggregation'),
            dcc.RadioItems(
                [*all_options.keys(),],
                'General',
                id='radio_district',
                inline=True,
                inputStyle={"margin-left": "20px"}),

            html.Label('District'),
            dcc.Dropdown(id='dropdown_district', value='City of London'),
            
            html.Label('Accident severity'),
            dcc.Checklist(accident_severity_options,
                      accident_severity_options,
                      id = 'checklist_severity',
                      inline=True,
                      inputStyle={"margin-left": "20px"}),
            
        ], style={'width': '48%', 'float': 'left', 'display': 'inline-block'}),
        
        # Right
        html.Div([
            html.Label('Unknown and missing data'),
            dcc.RadioItems(
                ['All data', 'Exclude'],
                'All data',
                id='radio_totals',
                inline=True,
                inputStyle={"margin-left": "20px"}),
            
            html.Label('Attribute'),
            dcc.Dropdown(
                [*attributes_dict.keys(),],
                'road_type',
                id='field_name'),          
            
            html.Label('Type of Vehicle'),
            dcc.Checklist(ev_options,
                          ev_options,
                          id = 'checklist_ev',
                          inline=True,
                          inputStyle={"margin-left": "20px"},),
            
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'}),

    ], className='row'),
    
    
    html.Div([  
        html.Div([
            dcc.Graph(id='map'),
            
            html.Div([
                dbc.Container(id='tbl_out'),
            ]),
        ], style={'width': '47%', 'float': 'left', 'display': 'inline-block'}),

        html.Div([
            dcc.Graph(id='indicator-graphic')
        ], style={'width': '47%', 'float': 'right', 'display': 'inline-block'}),
    ], className='row'),
    
    html.Br(),
    dcc.Link('Go to the Attributes correlation analysis', href='/attributes'),
    html.Br(),
    dcc.Link('Go back to the homepage', href='/')
])

# Setting the Map aggregation div
@app.callback(
    Output('dropdown_district', 'options'),
    Input('radio_district', 'value'))
def set_cities_options(selected_country):
    return all_options[selected_country]

@app.callback(
    Output('dropdown_district', 'value'),
    Input('dropdown_district', 'options'))
def set_cities_value(available_options):
    return available_options

@app.callback(
    Output('indicator-graphic', 'figure'),
    Output('map', 'figure'),
    Input('field_name', 'value'),
    Input('dropdown_district', 'value'),
    Input('radio_district', 'value'),
    Input('checklist_severity', 'value'),
    Input('checklist_ev', 'value'),
    Input('radio_totals', 'value'),
    Input('map', 'clickData'))
def update_graph(field_name, dropdown_district, radio_district, checklist_severity, checklist_ev, radio_totals, map_click): 
    # Data
    df = df_labeled
    xlsx = pd.read_excel('Road-Safety-Open-Dataset-Data-Guide.xlsx')
    hover_data = list(aggregations.keys())
    
    # EV Checkbox
    if len(checklist_ev) == 0: # if none is selected, select all
        checklist_ev = ev_options
    else:
        df = df.loc[df['ev'].isin(checklist_ev)]
    
    # Severity Checkbox
    if len(checklist_severity) == 0: # if none is selected, select all
        checklist_severity = accident_severity_options
    else:
        df = df.loc[df['accident_severity'].isin(checklist_severity)] 
    
    # By district or precise location:
    if radio_district == 'General':
        # Grouping districts by field_name
        df_group = df.groupby(['geo_code'])['number_of_casualties'].sum().to_frame()
        # Adding geografical coordinates
        df_group = df_group.merge(gdf, how='outer', left_on=['geo_code'], right_on=['geo_code']).dropna()
        # Input for the Attributes graph
        df_chart = df_labeled
        hover_name = 'name'
        zoom = 5
        hover_data = []
        title='Casualties: '+ field_name.replace('_',' ').capitalize()

    elif radio_district == 'Precise':
        df_chart = df[df['local_authority_ons_district']==dropdown_district]
        #df_group = df_chart.groupby(['longitude', 'latitude'])['number_of_casualties'].sum().to_frame().reset_index()
        df_group = df_chart.groupby(['longitude', 'latitude'], as_index=False).agg(aggregations)
        # Input for the Attributes graph
        hover_name = 'number_of_casualties'
        hover_data = list(aggregations.keys())
        zoom = 11
        title = dropdown_district.capitalize() + ' ' + 'casualties: '+ field_name.replace('_',' ').capitalize()

    # Map
    map_full = px.scatter_mapbox(df_group, lat="latitude", lon="longitude", color='number_of_casualties', size='number_of_casualties', mapbox_style = "open-street-map",
                          height = 700, zoom = zoom, opacity = 0.9, hover_name = hover_name, hover_data=hover_data, color_continuous_scale = plotly.colors.sequential.Purp)
    map_full.update_layout(clickmode='event+select')
    
    ### Attributes chart ###
    field_class = attributes_dict[field_name]
    
    hover_data = ['Percentage']
        
    # Frequency
    frequencies = pd.DataFrame(df_chart[field_name].unique(),columns=[field_name]).sort_values(by=field_name, ascending=True, ignore_index = True)
    for index, row in frequencies.iterrows():
        frequencies.loc[index,'Casualties'] = df_chart[df_chart[field_name]==row[0]]['number_of_casualties'].sum()
    if radio_totals == 'Exclude':
        frequencies = frequencies[~frequencies[field_name].isin(missing)]
    
    # Percentage    
    frequencies['Percentage'] = round(frequencies['Casualties']/frequencies['Casualties'].sum()*100,1)
    frequencies['Percentage'] = frequencies['Percentage'].apply(lambda x: str(x)+'%')  
    
    # Ordered (Ordinal) preparation
    if field_class == 'ordinal':
        # to read the xls with the correspondencies
        columns = ['code/format','label']
        # Spreadhseet with the code to label correspondence
        xlsx_df = xlsx[xlsx['field name'] == field_name][columns].drop_duplicates(subset='label',keep='first')
        frequencies = frequencies.merge(xlsx_df, how='left', left_on=[field_name], right_on=['label'])
        frequencies.sort_values(by=['code/format'], inplace=True)
    
    # Pie chart
    if frequencies.shape[0] <= 3:
        fig = px.pie(frequencies, values='Casualties', names=field_name, title=title)
        return  fig, map_full
        
    # Ordinal and quantitative
    if field_class != 'categorical':
        fig = px.line(frequencies, x=field_name, y='Casualties', hover_name = field_name, hover_data = hover_data, title = title)
        return fig, map_full
    
    # Categorical
    if field_class == 'categorical':
        frequencies.sort_values(by='Casualties', ascending=True, inplace = True)
        fig = px.bar(frequencies, x='Casualties', y=field_name, hover_name = field_name, hover_data = hover_data, title = title)
        return fig, map_full
    
@app.callback(
    Output('tbl_out', 'children'),
    Input('map', 'clickData'))
def display_click_data(clickData):
    district = dict(clickData['points'][0])['hovertext']
    df_table = df_address[df_address['local_authority_ons_district']== district]
    df_table.drop(columns=['local_authority_ons_district'], inplace=True)
    return dbc.Label(district), dash_table.DataTable(df_table.to_dict('records'),[{"name": i, "id": i} for i in df_table.columns], id='tbl')



### Page 2 ###

page_2_layout = html.Div([
    html.H1('Attributes'),
    html.Br(),
    dcc.Link('Go to the Road Casualties Map', href='/map'),
    html.Br(),
    dcc.Link('Go back to the homepage', href='/')
])




# Update the index
@callback(Output('homepage', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/map':
        return page_1_layout
    elif pathname == '/attributes':
        return page_2_layout
    else:
        return index_page
    # You could also return a 404 "URL not found" page here

if __name__ == '__main__':
    app.run_server(debug=True, port = 2077, use_reloader=False)

Dash app running on http://127.0.0.1:2077/


D:\Programs\anaconda3\lib\site-packages\jupyter_dash\jupyter_app.py:139: UserWarning:

The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.

